# DengAI: Predicting Disease Spread

Description: Using environmental data collected by various U.S. Federal Government agencies—from the Centers for Disease Control and Prevention to the National Oceanic and Atmospheric Administration in the U.S. Department of Commerce—can you predict the number of dengue fever cases reported each week in San Juan, Puerto Rico and Iquitos, Peru?

First, we'll take a look at some of the features of the training data

In [20]:
import pandas as pd # Data handling 
import numpy as np

import tensorflow as tf # Neural networks  
from tensorflow import keras
from tensorflow.keras import layers

import plotly.graph_objects as go # Visualization
import plotly.express as px

import sklearn as sk # Stats / ML
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV, TimeSeriesSplit
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import impute

from xgboost import XGBRegressor

In [21]:
df = pd.read_csv("../data/raw/dengue_features_train.csv")
df_labels = pd.read_csv("../data/raw/dengue_labels_train.csv")
df_features_test = pd.read_csv("../data/raw/dengue_features_test.csv")
submission_format = pd.read_csv("../data/raw/submission_format.csv")

Now, let's take a look at some visualizations of the data. The correlation matrix uses the Pearson correlation coefficient. More info on the documentation of .corr() can be found [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html)

In [22]:
df['total_cases'] = df_labels['total_cases'] #append labels to DataFrame 

# Seperate data for San Juan
sj_train_features = df.loc[df['city'] == 'sj']
sj_train_labels = sj_train_features['total_cases'].to_frame()

# Separate data for Iquitos
iq_train_features = df.loc[df['city'] == 'iq']
iq_train_labels = iq_train_features['total_cases'].to_frame()

df = pd.get_dummies(df, columns=['city'], drop_first=True)

# We don't need to encode city when we are seperating by city
sj_train_features.drop('city', axis=1, inplace=True)
iq_train_features.drop('city', axis=1, inplace=True) 

In [23]:
tc_fig = px.bar(df_labels, x='total_cases', title='Total Cases Distribution')
tc_fig.show()

time_fig = go.Figure(data=go.Scatter(
    x=sj_train_features['week_start_date'],
    y=sj_train_features['total_cases'],
    mode='lines', name='Total Cases for SJ'), 
    layout=go.Layout(xaxis_title='Date', yaxis_title='Number of Cases', title={
        'text':'Cases over Time for sj/iq'
    }),
)

time_fig.add_trace(go.Scatter(
    x=iq_train_features['week_start_date'],
    y=iq_train_features['total_cases'],
    mode='lines', name='Total Cases for IQ'))

time_fig.show()

fig_corr = go.Figure(data=go.Heatmap(
    z=df.corr(),
    x=df.corr().columns,
    y=df.corr().columns,
    hoverongaps=False,
), layout=go.Layout(
    height=900,
    width=900,
    title = {
        'text':'Correlation Matrix',
        'x':0.5,
        'xanchor':'center',
        'yanchor':'top'
    }

))

fig_corr.show()

charts = {'reanalysis_sat_precip_amt_mm':'Precipitation Amount (mm)', 'reanalysis_avg_temp_k':'Average Air Temperature (k)', 
        'reanalysis_relative_humidity_percent':'Mean Relative Humidity (%)',
        'reanalysis_dew_point_temp_k':'Average Dew Point Temperature (k)'
         }

for key in charts:
    data = go.Scatter(x=sj_train_features[key], y=sj_train_features['total_cases'], mode='markers', marker_color=sj_train_features['total_cases'])
    layout = go.Layout(
        title = {
            'text': charts[key] + ' vs Number of Cases',
            'y':0.9,
            'x':0.5,
            'xanchor':'center',
            'yanchor':'top'
        },
        xaxis_title = "Number of Cases",
        yaxis_title = charts[key]
    )
    fig = go.Figure(data=data, layout=layout)
    fig.show()

charts['total_cases'] = 'Total Cases'
fig = px.scatter_matrix(df, dimensions=list(charts))
fig.show()


From what we can see, the data is fairly Gaussian. The DewPoint data is slightly skewed, but we'll ignore that for now. Next, lets explore how clean the data is.

In [24]:
num_nans = sum([True for idx, row in df.iterrows() if any(row.isnull())]) #count number of rows with ANY NaN value
ratio_nan = num_nans / len(df)

print(str(ratio_nan * 100) + "% of rows have some NaN value\n\n")

d = go.Bar(x=df.columns, y=(df.isnull().sum() / df.shape[0]) * 100)
l = go.Layout(
    title = {
        'text': 'Number of NaN values (%) in columns',
        'y':0.9,
        'x':0.5,
        'xanchor':'center',
        'yanchor':'top'
    },
    xaxis_title = 'Column name',
    yaxis_title = 'Number of NaN values (%)'
)

fig = go.Figure(data=d, layout=l)
fig.show()

17.6510989010989% of rows have some NaN value




As we can see, the percentage of rows with some NaN value is pretty high, so it wouldn't be a great idea to just remove them entirely. Instead, lets try to fill those missing values with some meaningful data. 

I initially tested filling with the mean value, but that increased bias significantly. Let's try using scikit-learn's KNNImputer. The documentation and explanation can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html#sklearn.impute.IterativeImputer)

In [25]:
test = df.copy()

imp = impute.IterativeImputer() 
# imp = impute.KNNImputer()
imputed = imp.fit_transform(test[[i for i in df.columns if i != 'week_start_date']])
imputed_features = pd.DataFrame(imputed, index=test.index, columns=[i for i in test.columns if i != 'week_start_date'])
imputed_features['week_start_date'] = test['week_start_date']

df = imputed_features.copy()

# test = df_features_test.copy()
# imputed = imp.fit_transform(test[[i for i in df_features_test.columns if i != 'week_start_date' and i != 'city']])
# imputed_features = pd.DataFrame(imputed, index=test.index, columns=[i for i in test.columns if i != 'week_start_date' and i != 'city'])
# imputed_features['week_start_date'] = test['week_start_date']

# df_features_test = imputed_features.copy()


Now, lets split the dataset by cit and drop some highly correlated variables (by the Pearson correlation coefficient)

In [26]:
#Remove some highly correlated columns
df.drop(['precipitation_amt_mm', 'reanalysis_specific_humidity_g_per_kg', 'reanalysis_min_air_temp_k'], axis=1, inplace=True)
sj_train_features.drop(['precipitation_amt_mm', 'reanalysis_specific_humidity_g_per_kg', 'reanalysis_min_air_temp_k'], axis=1, inplace=True)
iq_train_features.drop(['precipitation_amt_mm', 'reanalysis_specific_humidity_g_per_kg', 'reanalysis_min_air_temp_k'], axis=1, inplace=True)



print(df.isnull().sum()) #should be zero for all columns

year                                    0
weekofyear                              0
ndvi_ne                                 0
ndvi_nw                                 0
ndvi_se                                 0
ndvi_sw                                 0
reanalysis_air_temp_k                   0
reanalysis_avg_temp_k                   0
reanalysis_dew_point_temp_k             0
reanalysis_max_air_temp_k               0
reanalysis_precip_amt_kg_per_m2         0
reanalysis_relative_humidity_percent    0
reanalysis_sat_precip_amt_mm            0
reanalysis_tdtr_k                       0
station_avg_temp_c                      0
station_diur_temp_rng_c                 0
station_max_temp_c                      0
station_min_temp_c                      0
station_precip_mm                       0
total_cases                             0
city_sj                                 0
week_start_date                         0
dtype: int64


Great. Now, lets try a constructing a baseline model using Linear Regression. Note that we do not have to normalize the data when using ordinary least squares. An explanation can be found [here](https://stats.stackexchange.com/questions/29781/when-conducting-multiple-regression-when-should-you-center-your-predictor-varia).

In [27]:
y = df_labels[['total_cases']]

df_no_y = df.copy()
df_no_y.drop(['total_cases', 'week_start_date'], inplace=True, axis=1)

kf = sk.model_selection.TimeSeriesSplit(n_splits=5) #Use sklearn TimeSeries CV to prevent fitting on "future" data
reg = linear_model.LinearRegression() # min(X(f(x)) - y)

train_errors = []
test_errors = []
diff_errors = []

for train_index, test_index in kf.split(df_no_y):
    d_train = df_no_y.iloc[train_index]
    t_train = y.iloc[train_index]
    
    d_test = df_no_y.iloc[test_index]
    t_test = y.iloc[test_index]
    
    reg.fit(d_train, t_train)
    pred_test = reg.predict(d_test)
    pred_train = reg.predict(d_train)
    train_errors.append(sk.metrics.mean_absolute_error(pred_train, t_train))
    test_errors.append(sk.metrics.mean_absolute_error(pred_test, t_test))

for tr, te in zip(train_errors, test_errors):
    diff_errors.append(tr - te)

So, we trained a linear regression model using CV with n=5 in order to reduce variance. Now, let's plot the train and test errors to get a rough estimate about our model's variance (overfitting).

In [28]:
fig = go.Figure()
train_trace = go.Scatter(y=train_errors, mode='lines', name='Train MAE')
test_trace = go.Scatter(y=test_errors, mode='lines', name='Test MAE')
diff_trace = go.Scatter(y=diff_errors, mode='lines', name='Train MAE - Test MAE')
fig.add_trace(train_trace)
fig.add_trace(test_trace)
fig.add_trace(diff_trace)
fig.update_layout(xaxis_title='i-th Fold', yaxis_title='MAE', title={
    'text':'Mean Absolute Errors',
    'y':0.9,
    'x':0.45,
    'xanchor':'center',
    'yanchor':'top'

})

fig.show()

def chart_actual_and_pred(df, df_to_pred, df_labels, model, mode, city_name):
    y_pred = model.predict(df_to_pred)
    trace_actual = go.Scatter(
        x=df['week_start_date'],
        y=df_labels['total_cases'],
        name='Actual Number of Cases',
        mode=mode
    )

    trace_pred = go.Scatter(
        x=df['week_start_date'],
        y=y_pred.ravel(),
        name='Predicted Number of Cases',
        mode=mode
    )

    pred_fig = go.Figure()
    pred_fig.add_trace(trace_actual)
    pred_fig.add_trace(trace_pred)
    pred_fig.update_layout(
        title='Predicted vs Actual Values for ' + city_name,
        xaxis_title='Date',
        yaxis_title='Total Cases',
    )

    pred_fig.show()

chart_actual_and_pred(df, df_no_y, df, reg, 'lines', 'Both Cities')

The model did not perform very well with Linear Regression, with an average MAE on the test set of about 23. However, note that we did not train each city seperately for this model. Let's treat this as our baseline.

The current leader on this competition has an MAE of about 10 on the test set, so we have a ways to go.

In [29]:
df.head()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,...,reanalysis_sat_precip_amt_mm,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,city_sj,week_start_date
0,1990.0,18.0,0.122600,0.103725,0.198483,0.177617,297.572857,297.742857,292.414286,299.8,...,12.42,2.628571,25.442857,6.900000,29.4,20.0,16.0,4.0,1.0,1990-04-30
1,1990.0,19.0,0.169900,0.142175,0.162357,0.155486,298.211429,298.442857,293.951429,300.9,...,22.82,2.371429,26.714286,6.371429,31.7,22.2,8.6,5.0,1.0,1990-05-07
2,1990.0,20.0,0.032250,0.172967,0.157200,0.170843,298.781429,298.878571,295.434286,300.5,...,34.54,2.300000,26.714286,6.485714,32.2,22.8,41.4,4.0,1.0,1990-05-14
3,1990.0,21.0,0.128633,0.245067,0.227557,0.235886,298.987143,299.228571,295.310000,301.4,...,15.36,2.428571,27.471429,6.771429,33.3,23.3,4.0,3.0,1.0,1990-05-21
4,1990.0,22.0,0.196200,0.262200,0.251200,0.247340,299.518571,299.664286,295.821429,301.9,...,7.52,3.014286,28.942857,9.371429,35.0,23.9,5.8,6.0,1.0,1990-05-28
